In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from funkagent import agents
from dotenv import load_dotenv

In [3]:
# Load config file
load_dotenv()

True

In [4]:
# create a simple function for testing with
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius.

    :param radius: The radius of the circle as a single, positive number (float) - no math calculations (*,+,-,/) as input allowed.
    :return: The circumference of the circle
    """
    return 2 * 3.14 * radius

In [16]:
agent = agents.Agent(
    functions=[circumference_calculator]
)

In [19]:
agent.ask("What is the circumference of a circle with a radius of 5?")

..

'The circumference of a circle with a radius of 5 is approximately 31.4 units.'

In [21]:
agent.internal_thoughts

[{'role': 'assistant',
  'function_call': <OpenAIObject at 0x10b1586d0> JSON: {
    "name": "circumference_calculator",
    "arguments": "{\n  \"radius\": 5\n}"
  },
  'content': None},
 {'role': 'function',
  'name': 'circumference_calculator',
  'content': '31.400000000000002'}]